<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/results/CMD_MCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install astro-datasets --upgrade
!pip install tensorflow_datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for astro-datasets: filename=astro_datasets-0.0.9-py3-none-any.whl size=15986 sha256=9975bb48b5655f1a257aeaffa3c64bf194c3fc0e64962c3e3d4335b8d26ba249
  Stored in directory: /root/.cache/pip/wheels/77/11/df/7eb516aa333b47883b39adabbe38508a09163be741b476dcda
Successfully built astro-datasets
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow_datasets as tfds
import astro_datasets
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

In [3]:
ds_train, info_train = tfds.load(name='cmd', split='train[:90%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot', 'parameters': ['omegam']})
ds_valid, info_valid = tfds.load(name='cmd', split='train[90%:95%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})
ds_test, info_test = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/I_Mtot_omegam/cmd/1.0.2.incompleteH1D0BC/cmd-train.tfrecord*...:   0%|    …

Dataset cmd downloaded and prepared to /root/tensorflow_datasets/I_Mtot_omegam/cmd/1.0.2. Subsequent calls will reuse this data.


In [4]:
minimum = np.array([0.1, 0.6])
maximum = np.array([0.5, 1.0])  



def normalize(image, label):  
  image = tf.math.log(image)
  label = (label - minimum[0])/(maximum[0] - minimum[0])
  return image, label



ds_train = ds_train.map(normalize)
ds_valid = ds_valid.map(normalize)
ds_test = ds_test.map(normalize)

def rotate_map(image, label):
    return tf.image.rot90(image, np.random.choice([-1,0,1,2])), label


#ds_train = ds_train.map(rotate_map)


In [5]:
class MonteCarloDropout(tf.keras.layers.Dropout):
     def call(self, inputs):
         return super().call(inputs, training=True)

In [6]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.RandomFlip(),
    tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, padding = 'same', input_shape=(256, 256, 1)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(26, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(32, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    tf.keras.layers.Flatten(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(120, activation='leaky_relu'),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(84),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(1, activation='leaky_relu'),
    ])

In [7]:
#model.summary()

In [8]:
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3)

In [9]:
model.compile(optimizer=opt,
              loss='MeanSquaredError',
              metrics='mae')


In [10]:
checkpoint_path = "cmd/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=False,
                                                  monitor='val_loss',
                                                  mode='min',
                                                  verbose=1,
                                                  save_best_only=True)

In [11]:

model.fit(ds_train, epochs=100, callbacks=[reduce_lr, cp_callback], validation_data=ds_valid)

Epoch 1/100
180/180 [==============================] - ETA: 0s - loss: 98.9314 - mae: 5.5632
Epoch 1: val_loss improved from inf to 4.22007, saving model to cmd/cp.ckpt
INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


180/180 [==============================] - 149s 722ms/step - loss: 98.9314 - mae: 5.5632 - val_loss: 4.2201 - val_mae: 1.7330 - lr: 5.0000e-05
Epoch 2/100
180/180 [==============================] - ETA: 0s - loss: 1.3132 - mae: 0.7912
Epoch 2: val_loss improved from 4.22007 to 0.14667, saving model to cmd/cp.ckpt
INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


180/180 [==============================] - 189s 1s/step - loss: 1.3132 - mae: 0.7912 - val_loss: 0.1467 - val_mae: 0.3115 - lr: 5.0000e-05
Epoch 3/100
180/180 [==============================] - ETA: 0s - loss: 0.1419 - mae: 0.3079
Epoch 3: val_loss did not improve from 0.14667
180/180 [==============================] - 117s 641ms/step - loss: 0.1419 - mae: 0.3079 - val_loss: 0.1503 - val_mae: 0.3143 - lr: 5.0000e-05
Epoch 4/100
180/180 [==============================] - ETA: 0s - loss: 0.1413 - mae: 0.3077
Epoch 4: val_loss improved from 0.14667 to 0.13899, saving model to cmd/cp.ckpt
INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


...

180/180 [==============================] - 127s 705ms/step - loss: 0.0843 - mae: 0.2508 - val_loss: 0.0851 - val_mae: 0.2523 - lr: 2.8440e-05
Epoch 63/100
180/180 [==============================] - ETA: 0s - loss: 0.0841 - mae: 0.2503
Epoch 63: val_loss did not improve from 0.08506
180/180 [==============================] - 113s 625ms/step - loss: 0.0841 - mae: 0.2503 - val_loss: 0.0874 - val_mae: 0.2562 - lr: 2.8440e-05
Epoch 64/100
180/180 [==============================] - ETA: 0s - loss: 0.0844 - mae: 0.2510
Epoch 64: val_loss did not improve from 0.08506
180/180 [==============================] - 112s 623ms/step - loss: 0.0844 - mae: 0.2510 - val_loss: 0.0860 - val_mae: 0.2536 - lr: 2.8440e-05
Epoch 65/100
180/180 [==============================] - ETA: 0s - loss: 0.0845 - mae: 0.2511
Epoch 65: val_loss did not improve from 0.08506
180/180 [==============================] - 113s 625ms/step - loss: 0.0845 - mae: 0.2511 - val_loss: 0.0860 - val_mae: 0.2534 - lr: 2.8440e-05
Epoch 66/

INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


180/180 [==============================] - 127s 703ms/step - loss: 0.0841 - mae: 0.2504 - val_loss: 0.0849 - val_mae: 0.2522 - lr: 2.5667e-05
Epoch 71/100
180/180 [==============================] - ETA: 0s - loss: 0.0841 - mae: 0.2505
Epoch 71: val_loss did not improve from 0.08492
180/180 [==============================] - 113s 624ms/step - loss: 0.0841 - mae: 0.2505 - val_loss: 0.0866 - val_mae: 0.2544 - lr: 2.5667e-05
Epoch 72/100
180/180 [==============================] - ETA: 0s - loss: 0.0839 - mae: 0.2503
Epoch 72: val_loss did not improve from 0.08492
180/180 [==============================] - 113s 625ms/step - loss: 0.0839 - mae: 0.2503 - val_loss: 0.0864 - val_mae: 0.2550 - lr: 2.5667e-05
Epoch 73/100
180/180 [==============================] - ETA: 0s - loss: 0.0842 - mae: 0.2505
Epoch 73: val_loss improved from 0.08492 to 0.08470, saving model to cmd/cp.ckpt
INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


INFO:tensorflow:Assets written to: cmd/cp.ckpt/assets


180/180 [==============================] - 127s 702ms/step - loss: 0.0842 - mae: 0.2505 - val_loss: 0.0847 - val_mae: 0.2520 - lr: 2.5667e-05
Epoch 74/100
180/180 [==============================] - ETA: 0s - loss: 0.0844 - mae: 0.2510
Epoch 74: val_loss did not improve from 0.08470
180/180 [==============================] - 113s 625ms/step - loss: 0.0844 - mae: 0.2510 - val_loss: 0.0863 - val_mae: 0.2542 - lr: 2.5667e-05
Epoch 75/100
180/180 [==============================] - ETA: 0s - loss: 0.0840 - mae: 0.2502
Epoch 75: val_loss did not improve from 0.08470
180/180 [==============================] - 113s 625ms/step - loss: 0.0840 - mae: 0.2502 - val_loss: 0.0864 - val_mae: 0.2549 - lr: 2.5667e-05
Epoch 76/100
180/180 [==============================] - ETA: 0s - loss: 0.0840 - mae: 0.2505
Epoch 76: val_loss did not improve from 0.08470
180/180 [==============================] - 113s 625ms/step - loss: 0.0840 - mae: 0.2505 - val_loss: 0.0863 - val_mae: 0.2550 - lr: 2.5667e-05
Epoch 77/

In [12]:
new_model = tf.keras.models.load_model('cmd/cp.ckpt')

In [13]:
def unnormalize(image, label):  
  image = tf.math.exp(image)
  label = (label*(maximum[0] - minimum[0])) + minimum[0]
  return image, label



In [14]:
for _ in range(10):
  acc = 0
  for (x, y) in ds_test:
    yhat = new_model.predict(x)
    yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
    y = (y*(maximum[0] - minimum[0])) + minimum[0]
    acc += np.mean(np.abs((yhat-y)/y))
    
  print('Inaccuracy: ', acc)

Inaccuracy:  4.494278896000954
Inaccuracy:  4.503241594851609
Inaccuracy:  4.508465931319103
Inaccuracy:  4.5463010695251835
Inaccuracy:  4.53080181249884
Inaccuracy:  4.550388407581038
Inaccuracy:  4.542334746029032
Inaccuracy:  4.51848798299786
Inaccuracy:  4.550157846484614
Inaccuracy:  4.505559354266792


In [15]:
ds_test_S, info_test_S = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'SIMBA', 'field': 'Mtot','parameters': ['omegam']})
ds_test_S = ds_test_S.map(normalize)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/S_Mtot_omegam/cmd/1.0.2.incompleteRJ0BTL/cmd-train.tfrecord*...:   0%|    …

Dataset cmd downloaded and prepared to /root/tensorflow_datasets/S_Mtot_omegam/cmd/1.0.2. Subsequent calls will reuse this data.


In [16]:
for _ in range(10):
  acc = 0
  for (x, y) in ds_test_S:
    yhat = new_model.predict(x)
    yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
    y = (y*(maximum[0] - minimum[0])) + minimum[0]
    acc += np.mean(np.abs((yhat-y)/y))
    
  print('Inaccuracy: ', acc)

Inaccuracy:  4.469021292679773
Inaccuracy:  4.457866109327513
Inaccuracy:  4.497693253293961
Inaccuracy:  4.497148581333669
Inaccuracy:  4.489340591825098
Inaccuracy:  4.458413497113211
Inaccuracy:  4.481682929770356
Inaccuracy:  4.4419404003987495
Inaccuracy:  4.474141032273019
Inaccuracy:  4.504957129983654
